In [3]:
import sqlite3
conn = sqlite3.connect('basketball_db')
cursor = conn.cursor()

In [ ]:
create_table_query = """create table cls (bname text, bage int); """
cursor.execute( create_table_query )

In [ ]:
cursor.execute("select * from cls;")
cursor.fetchall()
# Will Return Nothing

[('John Smith', 28),
 ('Sarah Johnson', 34),
 ('Michael Lee', 42),
 ('Emma Wilson', 25),
 ('David Brown', 31),
 ('John Smith', 28),
 ('Sarah Johnson', 34),
 ('Michael Lee', 42),
 ('Emma Wilson', 25),
 ('David Brown', 31)]

In [ ]:
insert_data_query = """
INSERT INTO cls (bname, bage) VALUES 
    ('John Smith', 28),
    ('Sarah Johnson', 34),
    ('Michael Lee', 42),
    ('Emma Wilson', 25),
    ('David Brown', 31)
;"""

cursor.execute(insert_data_query)

# commit --- why is this required?
conn.commit()

In [14]:
cursor.execute("select bage, bname from cls;")
print(cursor.description)
list_of_players = cursor.fetchall()

(('bage', None, None, None, None, None, None), ('bname', None, None, None, None, None, None))


In [26]:
def execute_query_return_list_of_dicts( conn, sql_query):

    cursor = conn.cursor()
    cursor.execute(sql_query)
    description_info = cursor.description
    
    headers = [x[0] for x in description_info]
    query_result = cursor.fetchall()
    
    return_dict_list = []
    
    for result in query_result:
        # single_result_dict = {headers[i] : result[i] for i in range(len(headers))}
        single_result_dict = dict(zip(headers, result))
    
        return_dict_list.append( single_result_dict )
    # Measure memory
    # 2 full copies of the data in memory: one in the query_resuls and one in the return_dict_list
    return return_dict_list

execute_query_return_list_of_dicts( conn, "select * from cls;")

[{'bname': 'John Smith', 'bage': 28},
 {'bname': 'Sarah Johnson', 'bage': 34},
 {'bname': 'Michael Lee', 'bage': 42},
 {'bname': 'Emma Wilson', 'bage': 25},
 {'bname': 'David Brown', 'bage': 31},
 {'bname': 'John Smith', 'bage': 28},
 {'bname': 'Sarah Johnson', 'bage': 34},
 {'bname': 'Michael Lee', 'bage': 42},
 {'bname': 'Emma Wilson', 'bage': 25},
 {'bname': 'David Brown', 'bage': 31}]

In [ ]:
def execute_query_return_list_of_dicts_lm( conn, sql_query):
    """
    Low memory version of pvs command
    """

    cursor = conn.cursor()
    cursor.execute(sql_query)
    description_info = cursor.description
    
    headers = [x[0] for x in description_info]
    return_dict_list = []

    while True:
        single_result = cursor.fetchone()

        if not single_result:
            break

        single_result_dict = dict(zip(headers, single_result))    
        return_dict_list.append( single_result_dict )

    return return_dict_list

execute_query_return_list_of_dicts_lm( conn, "select * from cls;")

[{'bname': 'John Smith', 'bage': 28},
 {'bname': 'Sarah Johnson', 'bage': 34},
 {'bname': 'Michael Lee', 'bage': 42},
 {'bname': 'Emma Wilson', 'bage': 25},
 {'bname': 'David Brown', 'bage': 31},
 {'bname': 'John Smith', 'bage': 28},
 {'bname': 'Sarah Johnson', 'bage': 34},
 {'bname': 'Michael Lee', 'bage': 42},
 {'bname': 'Emma Wilson', 'bage': 25},
 {'bname': 'David Brown', 'bage': 31},
 {'bname': 'a', 'bage': 5},
 {'bname': 'b', 'bage': 6},
 {'bname': 'c', 'bage': 7},
 {'bname': 'd', 'bage': 8},
 {'bname': 'e', 'bage': 9},
 {'bname': 'a', 'bage': 5},
 {'bname': 'b', 'bage': 6},
 {'bname': 'c', 'bage': 7},
 {'bname': 'd', 'bage': 8},
 {'bname': 'e', 'bage': 9}]

In [35]:
# Example of using execute many

insert_query_string = "insert into cls (bname, bage) values (? , ?)"

names = ['a', 'b', 'c', 'd', 'e']
ages = [5, 6, 7, 8, 9]

data_to_insert = list(zip(names, ages))


data_to_insert

cursor.executemany(insert_query_string, data_to_insert)

conn.commit()

In [ ]:
# Using indexes...
cursor.execute("create index idx_cls_bage on cls( bage );")
cursor.execute("create index idx_cls_bname_bage on cls( bname, bage);")

"select * from cls where bage = 2" <- index will be used!

"select * from cls where bname = 'Alice'" <- index will not be used!

"select * from cls where bname = 'Alice' and bage=2 " <- index may be used!

Strongly recommned: index on year and index of symbol
